In [6]:
import ast
import pandas as pd
# Simulated CSV content (string representation of a list of dictionaries)
# data_str = """[{'project name': 'FCCL', 'customer': 'Japanese Customer', 'position': 'Operation Monitoring', 'responsibilities': 'Write document and guiline task. Monitoring the fluency of the system. Doing daily task & task that required the correct date to execute. Log ticket on Redmine when problems occurs. Daily report', 'technologies used': ['Redmine', 'MS Office', 'MS SQL']}, {'project name': 'Cong ty TNHH dich vu 1PAYVi dien tu Trumoney', 'customer': 'Vietnamese Customer', 'position': 'System operator', 'responsibilities': 'System management, Network Management, Email management', 'technologies used': ['Grafana', 'Jira', 'MS Office', 'MS SQL', 'Linux']}, {'project name': 'Cong ty Co phan dich vu truc tuyen -Vi dien tu Momo', 'customer': 'Vietnamese Customer', 'position': 'Sale Staff', 'responsibilities': "Manage systems with retailer store. Co-operate with partner to run promote/events. Manage Momo's visible partners", 'technologies used': ['MS Office']}]"""
df = pd.read_csv('program.csv')
data_str = df.iloc[0]['Projects Experience']
# Convert the string to a list of dictionaries
projects = ast.literal_eval(data_str)

cypher_queries = []

for project in projects:
    project_name = project.get('project name', '').replace("'", "\\'")
    customer_name = project.get('customer', '').replace("'", "\\'")
    position = project.get('position', '').replace("'", "\\'")
    responsibilities = project.get('responsibilities', '').replace("'", "\\'")
    technologies = project.get('technologies used', [])

    # Create a string representation of the technology list for the Cypher query
    tech_list = "[" + ", ".join(f"'{tech}'" for tech in technologies) + "]"

    query = (
        f"WITH {tech_list} AS techList\n"
        f"MERGE (p:Project {{name: '{project_name}'}})\n"
        f"SET p.position = '{position}', p.responsibilities = '{responsibilities}'\n"
        f"MERGE (c:Customer {{name: '{customer_name}'}})\n"
        f"MERGE (p)-[:FOR_CUSTOMER]->(c)\n"
        f"WITH WITH p,techList\n"
        f"UNWIND techList AS tech\n"
        f"MERGE (t:Technology {{name: tech}})\n"
        f"MERGE (p)-[:USES_TECHNOLOGY]->(t)\n"
    )
    cypher_queries.append(query)

# Print the generated Cypher queries
for q in cypher_queries:
    print(q)


WITH ['Redmine', 'MS Office', 'MS SQL'] AS techList
MERGE (p:Project {name: 'FCCL'})
SET p.position = 'Operation Monitoring', p.responsibilities = 'Write document and guiline task. Monitoring the fluency of the system. Doing daily task & task that required the correct date to execute. Log ticket on Redmine when problems occurs. Daily report'
MERGE (c:Customer {name: 'Japanese Customer'})
MERGE (p)-[:FOR_CUSTOMER]->(c)
WITH WITH p,techList
UNWIND techList AS tech
MERGE (t:Technology {name: tech})
MERGE (p)-[:USES_TECHNOLOGY]->(t)

WITH ['Grafana', 'Jira', 'MS Office', 'MS SQL', 'Linux'] AS techList
MERGE (p:Project {name: 'Cong ty TNHH dich vu 1PAYVi dien tu Trumoney'})
SET p.position = 'System operator', p.responsibilities = 'System management, Network Management, Email management'
MERGE (c:Customer {name: 'Vietnamese Customer'})
MERGE (p)-[:FOR_CUSTOMER]->(c)
WITH WITH p,techList
UNWIND techList AS tech
MERGE (t:Technology {name: tech})
MERGE (p)-[:USES_TECHNOLOGY]->(t)

WITH ['MS Off